In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [2]:
class ElectricityHourPrice:
    def __init__(self, 
                 date, 
                 price_zone_code, 
                 consumer_volume, 
                 consumer_price, 
                 consumer_rd_volume,
                 consumer_spot_volume,
                 consumer_provide_rd,
                 consumer_max_price,
                 consumer_min_price,
                 supplier_volume,
                 supplier_price,
                 supplier_rd_volume,
                 supplier_spot_volume,
                 supplier_provide_rd,
                 supplier_max_price,
                 supplier_min_price,
                 hour):
        self.date = date
        self.price_zone_code = price_zone_code
        self.consumer_volume = consumer_volume
        self.consumer_price = consumer_price
        self.consumer_rd_volume = consumer_rd_volume
        self.consumer_spot_volume = consumer_spot_volume
        self.consumer_provide_rd = consumer_provide_rd
        self.consumer_max_price = consumer_max_price
        self.consumer_min_price = consumer_min_price
        self.supplier_volume = supplier_volume
        self.supplier_price = supplier_price
        self.supplier_rd_volume = supplier_rd_volume
        self.supplier_spot_volume = supplier_spot_volume
        self.supplier_provide_rd = supplier_provide_rd
        self.supplier_max_price = supplier_max_price
        self.supplier_min_price = supplier_min_price
        self.hour = hour

In [3]:
import requests
import xmltodict
from operator import itemgetter

def get_prices_by_date(date, price_zone = '2'):
    f_date = date.strftime('%Y%m%d')
    url = f'https://www.atsenergo.ru/market/stats.xml?period=0&date1={f_date}&date2={f_date}&zone={price_zone}&type=graph'
    response = requests.get(url, verify = False) #verify = False --> self-signed cert
    content = response.content
    result = xmltodict.parse(content)
    rows = list(map(itemgetter('col'), result['view']['row']))
    return rows

In [4]:
import datetime

start_date = datetime.datetime(2024, 5, 28)
end_date = datetime.datetime(2024, 6, 20)

print(f'Start date: {start_date.strftime("%Y-%m-%d")}')
print(f'End date: {end_date.strftime("%Y-%m-%d")}')
print(f'Time period: {str(end_date - start_date)}')

Start date: 2024-05-28
End date: 2024-06-20
Time period: 23 days, 0:00:00


In [5]:
import warnings
warnings.filterwarnings("ignore")

price_zone = '2'

electricity_hour_prices = []
cur_date = start_date
print(f"Starting getting prices for {start_date.strftime('%Y-%m-%d')}")
while cur_date <= end_date:
    if (cur_date.day % 7 == 0):
        print(f"Getting prices for {cur_date.strftime('%Y-%m-%d')}")
    try:
        price_rows = get_prices_by_date(cur_date, price_zone = price_zone)
        for price_row in price_rows:
            electricity_hour_prices.append(ElectricityHourPrice(*price_row))
    except Exception as e:
        print(f"Can't get the price data for {cur_date.strftime('%Y-%m-%d')}")
        print(f"Reason: {e}")
    finally:
        cur_date += datetime.timedelta(days = 1)
print(f"Finished getting prices for {end_date.strftime('%Y-%m-%d')}")

Starting getting prices for 2024-05-28
Getting prices for 2024-05-28
Getting prices for 2024-06-07
Getting prices for 2024-06-14
Finished getting prices for 2024-06-20


In [6]:
df = pd.DataFrame([vars(price_row) for price_row in electricity_hour_prices])
df

,date,price_zone_code,consumer_volume,consumer_price,consumer_rd_volume,consumer_spot_volume,consumer_provide_rd,consumer_max_price,consumer_min_price,supplier_volume,supplier_price,supplier_rd_volume,supplier_spot_volume,supplier_provide_rd,supplier_max_price,supplier_min_price,hour
0,28.05.2024,2,21985.908,534.79,1668.294,20105.932,118.844,1186.46,0.00,22948.207,422.41,1668.294,20627.933,69.820,6356.46,0.00,0
1,28.05.2024,2,22232.647,381.10,1753.041,20334.882,165.558,1120.58,0.00,23210.744,302.78,1753.041,20805.257,69.354,6356.46,0.00,1
2,28.05.2024,2,22907.350,883.74,4475.091,18256.510,128.579,1264.94,698.06,23809.856,834.76,4475.091,18929.606,316.641,6356.46,698.06,2
3,28.05.2024,2,23722.423,987.04,4508.715,19045.335,124.092,1288.62,794.18,24576.172,938.87,4508.715,19662.823,317.166,6356.46,794.18,3
4,28.05.2024,2,24427.048,1011.05,3768.499,20479.524,99.388,1373.18,747.67,25389.722,928.28,3768.499,21095.995,203.422,6356.46,747.67,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,20.06.2024,2,23345.975,1069.27,6168.354,16956.643,56.345,1702.69,787.36,24241.013,965.68,6168.354,17853.626,485.317,6375.03,787.36,19
572,20.06.2024,2,22643.250,1113.81,1352.804,21000.840,12.790,1659.41,787.36,23535.944,1020.08,1352.804,21511.823,33.033,6375.03,787.36,20
573,20.06.2024,2,22309.113,1070.62,1352.804,20675.408,27.543,1563.00,787.36,23332.529,989.34,1352.804,21308.408,33.033,6375.03,787.36,21
574,20.06.2024,2,21868.219,943.40,1352.804,20225.772,38.963,1311.14,757.78,22834.950,887.99,1352.804,20810.829,33.033,6375.03,757.78,22


In [7]:
df_form = df.copy()

In [8]:
df_form[df.columns[1:-1]] = df_form[df.columns[1:-1]].astype(float)
df_form['hour'] = df_form['hour'].astype(int)
df_form['price_zone_code'] = df_form['price_zone_code'].astype(int)

In [9]:
dates_from_table = df_form['date']
hours_from_table = df_form['hour']

In [10]:
from datetime import datetime

dates = [datetime.strptime(i, '%d.%m.%Y') for i in dates_from_table]
dates = [item[0].replace(hour = item[1]) for item in zip(dates, hours_from_table)]

In [11]:
df_form.drop(columns = ['date', 'hour'], inplace = True)
df_form['date'] = dates
df_form['date'] = pd.to_datetime(df_form['date'], utc = True)
df_form = df_form.set_index('date')
df_form = df_form.asfreq('1H')
df_form.sort_index(inplace = True)
df_form

,price_zone_code,consumer_volume,consumer_price,consumer_rd_volume,consumer_spot_volume,consumer_provide_rd,consumer_max_price,consumer_min_price,supplier_volume,supplier_price,supplier_rd_volume,supplier_spot_volume,supplier_provide_rd,supplier_max_price,supplier_min_price
date,,,,,,,,,,,,,,,
2024-05-28 00:00:00+00:00,2,21985.908,534.79,1668.294,20105.932,118.844,1186.46,0.00,22948.207,422.41,1668.294,20627.933,69.820,6356.46,0.00
2024-05-28 01:00:00+00:00,2,22232.647,381.10,1753.041,20334.882,165.558,1120.58,0.00,23210.744,302.78,1753.041,20805.257,69.354,6356.46,0.00
2024-05-28 02:00:00+00:00,2,22907.350,883.74,4475.091,18256.510,128.579,1264.94,698.06,23809.856,834.76,4475.091,18929.606,316.641,6356.46,698.06
2024-05-28 03:00:00+00:00,2,23722.423,987.04,4508.715,19045.335,124.092,1288.62,794.18,24576.172,938.87,4508.715,19662.823,317.166,6356.46,794.18
2024-05-28 04:00:00+00:00,2,24427.048,1011.05,3768.499,20479.524,99.388,1373.18,747.67,25389.722,928.28,3768.499,21095.995,203.422,6356.46,747.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-20 19:00:00+00:00,2,23345.975,1069.27,6168.354,16956.643,56.345,1702.69,787.36,24241.013,965.68,6168.354,17853.626,485.317,6375.03,787.36
2024-06-20 20:00:00+00:00,2,22643.250,1113.81,1352.804,21000.840,12.790,1659.41,787.36,23535.944,1020.08,1352.804,21511.823,33.033,6375.03,787.36
2024-06-20 21:00:00+00:00,2,22309.113,1070.62,1352.804,20675.408,27.543,1563.00,787.36,23332.529,989.34,1352.804,21308.408,33.033,6375.03,787.36


In [12]:
def verify_index(df):
    try:
        return (df.index == pd.date_range(start = df.index.min(), end = df.index.max(), freq = df.index.freq)).all()
    except:
        print("Error: can't verify temporary index")
        return False

verify_index(df_form)

True

In [13]:
TARGET_COLUMN = 'supplier_price'
df = df_form.copy()
df = df[[TARGET_COLUMN]]
df

,supplier_price
date,
2024-05-28 00:00:00+00:00,422.41
2024-05-28 01:00:00+00:00,302.78
2024-05-28 02:00:00+00:00,834.76
2024-05-28 03:00:00+00:00,938.87
2024-05-28 04:00:00+00:00,928.28
...,...
2024-06-20 19:00:00+00:00,965.68
2024-06-20 20:00:00+00:00,1020.08
2024-06-20 21:00:00+00:00,989.34


In [14]:
def add_time_features(df):
    df['year'] = df.index.year
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['week_of_year'] = pd.Index(df.index.isocalendar().week)
    df['week_of_year'] = df['week_of_year'].astype(int)
    df['day'] = df.index.day
    df['day_of_year'] = df.index.dayofyear
    df['day_of_week'] = df.index.dayofweek
    df['hour'] = df.index.hour
    
add_time_features(df)
df = df[['year', 'quarter', 'month', 'week_of_year', 'day_of_year', 'day_of_week', 'day', 'hour', TARGET_COLUMN]]
df

,year,quarter,month,week_of_year,day_of_year,day_of_week,day,hour,supplier_price
date,,,,,,,,,
2024-05-28 00:00:00+00:00,2024,2,5,22,149,1,28,0,422.41
2024-05-28 01:00:00+00:00,2024,2,5,22,149,1,28,1,302.78
2024-05-28 02:00:00+00:00,2024,2,5,22,149,1,28,2,834.76
2024-05-28 03:00:00+00:00,2024,2,5,22,149,1,28,3,938.87
2024-05-28 04:00:00+00:00,2024,2,5,22,149,1,28,4,928.28
...,...,...,...,...,...,...,...,...,...
2024-06-20 19:00:00+00:00,2024,2,6,25,172,3,20,19,965.68
2024-06-20 20:00:00+00:00,2024,2,6,25,172,3,20,20,1020.08
2024-06-20 21:00:00+00:00,2024,2,6,25,172,3,20,21,989.34


In [15]:
df.to_csv(f'electricity_price_hourly_{price_zone}_zone_supplier_{start_date.strftime("%Y-%m-%d")}__{end_date.strftime("%Y-%m-%d")}.csv', index = True)